First Cell to import important packages

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix
import time
from datetime import timedelta
import math
import os

# Use PrettyTensor to simplify Neural Network construction.
import prettytensor as pt

import cifar10
from cifar10 import img_size, num_channels, num_classes
img_size_cropped = 24

In [2]:
x = tf.placeholder(tf.float32, shape=[None, img_size, img_size, num_channels], name='x')

In [3]:
y_true = tf.placeholder(tf.float32, shape=[None, num_classes], name='y_true')

In [4]:
y_true_cls = tf.argmax(y_true, dimension=1)

In [5]:
def main_network(images, training):
    # Wrap the input images as a Pretty Tensor object.
    x_pretty = pt.wrap(images)

    # Pretty Tensor uses special numbers to distinguish between
    # the training and testing phases.
    if training:
        phase = pt.Phase.train
    else:
        phase = pt.Phase.infer

    # Create the convolutional neural network using Pretty Tensor.
    # It is very similar to the previous tutorials, except
    # the use of so-called batch-normalization in the first layer.
    with pt.defaults_scope(activation_fn=tf.nn.relu, phase=phase):
        y_pred= x_pretty.\
            conv2d(kernel=5, depth=64, name='layer_conv1', batch_normalize=True).\
            max_pool(kernel=2, stride=2).\
            conv2d(kernel=3, depth=64, name='layer_conv2', batch_normalize=True).\
            max_pool(kernel=2, stride=2).\
            flatten().\
            fully_connected(size=512, name='layer_fc1').\
            fully_connected(size=256, name='layer_fc2').\
            fully_connected(size=128, name='layer_fc3').\
            softmax_classifier(num_classes=num_classes, labels=y_true)

    return y_pred
#(N-F)/S+1 output size!

In [6]:
def create_network(training):
    # Wrap the neural network in the scope named 'network'.
    # Create new variables during training, and re-use during testing.
    with tf.variable_scope('network', reuse=not training):
        # Just rename the input placeholder variable for convenience.
        images = x

        # Create TensorFlow graph for pre-processing.
        images = pre_process(images=images, training=training)

        # Create TensorFlow graph for the main processing.
        y_pred= main_network(images=images, training=training)

    return y_pred

In [7]:
def pre_process_image(image, training):
    # This function takes a single image as input,
    # and a boolean whether to build the training or testing graph.
    
    if training:
        # For training, add the following to the TensorFlow graph.

        # Randomly crop the input image.
        image = tf.random_crop(image, size=[img_size_cropped, img_size_cropped, num_channels])

        # Randomly flip the image horizontally.
        image = tf.image.random_flip_left_right(image)
        
        # Randomly adjust hue, contrast and saturation.
        image = tf.image.random_hue(image, max_delta=0.05)
        image = tf.image.random_contrast(image, lower=0.3, upper=1.0)
        image = tf.image.random_brightness(image, max_delta=0.2)
        image = tf.image.random_saturation(image, lower=0.0, upper=2.0)

        # Some of these functions may overflow and result in pixel
        # values beyond the [0, 1] range. It is unclear from the
        # documentation of TensorFlow 0.10.0rc0 whether this is
        # intended. A simple solution is to limit the range.

        # Limit the image pixels between [0, 1] in case of overflow.
        image = tf.minimum(image, 1.0)
        image = tf.maximum(image, 0.0)
    else:
        #For training, add the following to the TensorFlow graph.

        #Crop the input image around the centre so it is the same
        #size as images that are randomly cropped during training.
        image = tf.image.resize_image_with_crop_or_pad(image,
                                                       target_height=img_size_cropped,
                                                       target_width=img_size_cropped)

    return image

In [8]:
def pre_process(images, training):
    # Use TensorFlow to loop over all the input images and call
    # the function above which takes a single image as input.
    images = tf.map_fn(lambda image: pre_process_image(image, training), images)

    return images

In [9]:
class_names = cifar10.load_class_names()
#type(class_names)
class_names

Loading data: data/CIFAR-10/cifar-10-batches-py/batches.meta


['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']

In [10]:
global_step = tf.Variable(initial_value=0,name='global_step', trainable=False)

In [11]:
print(global_step)

Tensor("global_step/read:0", shape=(), dtype=int32)


In [12]:
#tf.get_variable_scope().reuse_variables()

In [13]:
y_pred = create_network(training = False)

ValueError: Variable network/layer_conv1/weights does not exist, or was not created with tf.get_variable(). Did you mean to set reuse=None in VarScope?

In [ ]:
y_pred_cls = tf.argmax(y_pred, dimension=1)

In [ ]:
saver = tf.train.Saver()

In [ ]:
sess = tf.Session()

In [ ]:
#sess.run(tf.global_variables_initializer())

In [ ]:
save_dir = 'checkpoints/'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
save_path = save_dir + 'cifar10_cnn'


In [ ]:
try:
    print("Trying to restore last checkpoint ...")
    new_saver = tf.train.Saver()
    # Use TensorFlow to find the latest checkpoint - if any.
    last_chk_path = tf.train.latest_checkpoint(checkpoint_dir=save_dir)
    
    print(last_chk_path)
    
    # Try and load the data in the checkpoint.
    new_saver = tf.train.import_meta_graph(last_chk_path+".meta")
    
    #saver.restore(sess, save_path=last_chk_path)
    
    
    # If we get to this point, the checkpoint was successfully loaded.
    print("Restored checkpoint from:", last_chk_path)
except:
    # If the above failed for some reason, simply
    # initialize all the variables for the TensorFlow graph.
    print("Failed to restore checkpoint. Initializing variables instead.")
    sess.run(tf.global_variables_initializer())

In [ ]:
def plot_originalImg_and_32pImg(original_Img,Img32):
    plt.rcParams['figure.figsize'] = (10.0,10.0)

    fig, axes = plt.subplots(1, 2)

    # References to the sub-plots.
    ax0 = axes.flat[0]
    ax1 = axes.flat[1]

    ax0.imshow(original_Img)
    ax1.imshow(Img32)

    # Set labels.
    ax0.set_xlabel('original image')
    ax1.set_xlabel('image after traitment')
    plt.show()

In [ ]:
def predict_one_image(image):
    #np.set_printoptions(precision=3, suppress=True)
    
    prediction ,indice = sess.run([y_pred, y_pred_cls],feed_dict={x: [image]})
    #label_pred, cls_pred = session.run([y_pred, y_pred_cls],feed_dict={x: [img]}
    
    #print(prediction[0])        #show the output tensor in onehot incoded 
    #print(indice[0])            #shows the indice of the predicted class 
    
    return class_names[indice[0]],prediction[0]

In [ ]:
def Show_prediction(Img):
    try:
        np.set_printoptions(precision=3, suppress=True)

        class_name,_vect = predict_one_image(Img)

        plt.rcParams['figure.figsize'] = (4,4)
        plt.title(class_name)

        plt.imshow(myImage)

        plt.rcParams['figure.figsize'] = (1.0,1.0) #for real dimension
        plt.figure()
        plt.title("the real classified image")
        plt.imshow(Img)

        #function img.getbbox() returns picture dimensions

    except ValueError:
        print("There is a problem fix it !!")

In [ ]:
#import an image from your machine!!

from PIL import Image
#myImage = Image.open("images/dog4.jpg")

def import_resize_predict_image(myImage):
    size32 = (32,32)

    print("the original size :",myImage.getbbox())  #the original size of the image
    original = myImage.copy()
    
    w,h = myImage.size

    if w==h:
        lastImg = myImage.copy()
        lastImg.thumbnail(size32)
        
        #called while we were debugging
            #print("\nwidth EQUAL height\n")
            #print("size after thumbnail",myImage.getbbox())
        #called while we were debugging
    elif w>h:
        lastImg = myImage.crop((0, 0, w,w))
        lastImg.thumbnail((size32))
        
        #called while we were debugging
            #print("\nWidth LARGE\n")
            #print("size after thumbnail",lastImg.getbbox())
            #lastImg.save("images/32Img.jpg")
            #print("myImage :",myImage.getbbox())
            #print("lastImg :",lastImg.getbbox())
        #called while we were debugging
    else:
        lastImg = myImage.crop((0, 0, h,h))
        lastImg.thumbnail((size32))
        
        #called while we were debugging
            #print("\nHeight LARGE\n")
            #print("size after thumbnail",lastImg.getbbox())
            #print("myImage :",myImage.getbbox())
            #print("lastImg :",lastImg.getbbox())
        #called while we were debugging

        
    #plot_originalImg_and_32pImg(original,lastImg)  #just showing the original image next to real clasified image
    Img = np.asarray(lastImg)   #converting the ready for predicition image
    
    Show_prediction(Img)
    #print(Img.shape)
    #print(Img.size)
    #print(Img.dtype)
    #print(type(Img))
    #myImage.resize((32,32),Image.ANTIALIAS).save("voiture1_32.jpg")

In [ ]:
#img, cls = get_test_image(897)   #this get an image and label from test images from cifar data base 
#print(Img.shape)                 #image shape is knows as (32, 32, 3)

myImage = Image.open("images/dog4.jpg")
import_resize_predict_image(myImage)

JUST TO SHOW ACCURACY !!!!


In [ ]:
def classification_accuracy(correct):
    # When averaging a boolean array, False means 0 and True means 1.
    # So we are calculating: number of True / len(correct) which is
    # the same as the classification accuracy.
    
    # Return the classification accuracy
    # and the number of correct classifications.
    return correct.mean(), correct.sum()

In [ ]:
def print_test_accuracy(show_example_errors=False,
                        show_confusion_matrix=False):

    # For all the images in the test-set,
    # calculate the predicted classes and whether they are correct.
    correct, cls_pred = predict_cls_test()
    
    # Classification accuracy and the number of correct classifications.
    acc, num_correct = classification_accuracy(correct)
    
    # Number of images being classified.
    num_images = len(correct)

    # Print the accuracy.
    msg = "Accuracy on Test-Set: {0:.1%} ({1} / {2})"
    print(msg.format(acc, num_correct, num_images))

    # Plot some examples of mis-classifications, if desired.
    if show_example_errors:
        print("Example errors:")
        plot_example_errors(cls_pred=cls_pred, correct=correct)

    # Plot the confusion matrix, if desired.
    if show_confusion_matrix:
        print("Confusion Matrix:")
        plot_confusion_matrix(cls_pred=cls_pred)

In [ ]:
batch_size = 256

def predict_cls(images, labels, cls_true):
    # Number of images.
    num_images = len(images)

    # Allocate an array for the predicted classes which
    # will be calculated in batches and filled into this array.
    cls_pred = np.zeros(shape=num_images, dtype=np.int)

    # Now calculate the predicted classes for the batches.
    # We will just iterate through all the batches.
    # There might be a more clever and Pythonic way of doing this.

    # The starting index for the next batch is denoted i.
    i = 0

    while i < num_images:
        # The ending index for the next batch is denoted j.
        j = min(i + batch_size, num_images)

        # Create a feed-dict with the images and labels
        # between index i and j.
        feed_dict = {x: images[i:j, :],y_true: labels[i:j, :]}

        # Calculate the predicted class using TensorFlow.
        cls_pred[i:j] = sess.run(y_pred_cls, feed_dict=feed_dict)

        # Set the start-index for the next batch to the
        # end-index of the current batch.
        i = j

    # Create a boolean array whether each image is correctly classified.
    correct = (cls_true == cls_pred)

    return correct, cls_pred

In [ ]:
images_test, cls_test, labels_test = cifar10.load_test_data()

In [ ]:
def predict_cls_test():
    return predict_cls(images = images_test,
                       labels = labels_test,
                       cls_true = cls_test)

In [ ]:
print_test_accuracy(show_example_errors=False,show_confusion_matrix=False)